In [1]:
import redis
import json

r = redis.Redis(host="redis")

# Save and find single users

Let's create two functions to save and get the user:

In [2]:
def save_user(user):
  value = json.dumps(user)
  key = f"users:{user['id']}"
  return r.set(key, value)


def find_user(id):
  key = f"users:{id}"
  value = r.get(key)
  if value is None:
    return False
  return json.loads(value)

Let's create two users and save them:

In [3]:
bob = {'id': 8, 'name': "bob", 'born': 1986, 'country': "DE"}
alice = {'id': 12, 'name': "alice", 'born': 1995, 'country': "DE"}

In [4]:
save_user(bob)

True

In [5]:
save_user(alice)

True

Now, we can try to fetch a user with an invalid id:

In [6]:
find_user(9)

False

Time to fetch them with a valid id:

In [7]:
find_user(8)

{'id': 8, 'name': 'bob', 'born': 1986, 'country': 'DE'}

In [8]:
find_user(12)

{'id': 12, 'name': 'alice', 'born': 1995, 'country': 'DE'}

# Find all users

Currently, there is no way to get a list of all users, let's store an index with all users and a corresponding function to find all users:

In [9]:
def save_user(user):
  # single user
  value = json.dumps(user)
  key = f"users:{user['id']}"
  r.set(key, value)
  
  # add to index
  return r.sadd("user_ids", user['id'])


def find_users():
  # get list of ids
  ids = r.smembers("user_ids")
  
  # build user keys
  keys = [f"users:{id.decode('utf8')}" for id in ids]

  # multi-get all keys at once
  values = r.mget(keys)

  # parse responses into user
  return [json.loads(value) for value in values]

To add users to our index, we need to resave them:

In [10]:
save_user(bob)
save_user(alice)

0

Now, we can find all users:

In [11]:
find_users()

[{'id': 8, 'name': 'bob', 'born': 1986, 'country': 'DE'},
 {'id': 12, 'name': 'alice', 'born': 1995, 'country': 'DE'}]

# Add project entity

Let's add a project entity, we cann associate users with:

In [12]:
def save_project(project):
  # single project
  value = json.dumps(project)
  key = f"projects:{project['id']}"
  r.set(key, value)
  
  # add to index
  return r.sadd("project_ids", project['id'])

And now add a few projects:

In [13]:
website = { 'id': 234, 'name': 'Website' }
bycicle = { 'id': 235, 'name': 'Bycicle' }
shop = { 'id': 8362, 'name': 'Shop' }

save_project(website)
save_project(bycicle)
save_project(shop)

0

To add users to a project, we create two functions:

In [14]:
def add_user_to_project(user, project):
    return r.sadd(f"project:{project['id']}:user_ids", user['id'])

def find_project_users(project):
  # get list of ids
  ids = r.smembers(f"project:{project['id']}:user_ids")
  
  # build user keys
  keys = [f"users:{id.decode('utf8')}" for id in ids]

  # multi-get all keys at once
  values = r.mget(keys)

  # parse responses into user
  return [json.loads(value) for value in values]

Now, we can add users to projects and find them again:

In [15]:
add_user_to_project(alice, website)

0

In [16]:
find_project_users(website)

[{'id': 8, 'name': 'bob', 'born': 1986, 'country': 'DE'},
 {'id': 12, 'name': 'alice', 'born': 1995, 'country': 'DE'}]

In [17]:
add_user_to_project(bob, website)

0

In [18]:
find_project_users(website)

[{'id': 8, 'name': 'bob', 'born': 1986, 'country': 'DE'},
 {'id': 12, 'name': 'alice', 'born': 1995, 'country': 'DE'}]